# TODO: Write Python code to design an investment analysis chatbot using the Microsoft's AutoGen Framework.

(include a Google Colab link for submission)


notes:

- use print log instead log file because need to sent via ggcolab
- use class for deploy in .py but need to implement in colab
- no caching apikey, log new access every time


## Setup


In [1]:
# Check the environment
!conda env list


# conda environments:
#
base                   /Users/pupipatsingkhorn/miniconda3
datascience          * /Users/pupipatsingkhorn/miniconda3/envs/datascience



In [2]:
# Install libraries
%pip install "autogen-agentchat" "autogen-ext[openai]"
%pip install nest-asyncio

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


## Import


In [3]:
# # Standard library imports
import os

from typing import Dict
from datetime import datetime
from dotenv import load_dotenv


# # Third-party imports
import pandas as pd
import yfinance as yf
import asyncio
import nest_asyncio

# Microsoft's AutoGen imports
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.teams import SelectorGroupChat
from autogen_ext.models.openai import OpenAIChatCompletionClient

In [4]:
class Logger:
    """Logger class for handling different types of log messages."""

    def __init__(self, module_name: str):
        """
        Initialize logger with module name.

        Args:
            module_name: Name of the module using the logger
        """
        self.module_name = module_name

    def info(self, message: str) -> None:
        """Log informational messages with timestamp."""
        timestamp = datetime.now().isoformat()
        print(f"[{timestamp}] [INFO] [{self.module_name}] {message}")

    def error(self, message: str) -> None:
        """Log error messages with timestamp."""
        timestamp = datetime.now().isoformat()
        print(f"[{timestamp}] [ERROR] [{self.module_name}] {message}")


class Config:
    """
    Configuration class for managing API credentials and model settings.
    """

    MODEL_NAME: str = "gemini-2.0-flash"
    API_KEY_NAME: str = "GEMINI_API_KEY"

    def __init__(self):
        """Initialize configuration and load environment variables."""
        self.logger = Logger("Config")
        self._load_environment()

    def _load_environment(self) -> None:
        """Load environment variables from .env file."""
        load_dotenv()
        self.logger.info("Environment variables loaded")

    def get_api_key(self) -> str:
        """
        Securely retrieves the API key from the environment variable.

        Returns:
            str: The API key.

        Raises:
            ValueError: If API key is not set in environment variables.
        """
        self.logger.info(f"Attempting to access {self.API_KEY_NAME}")
        api_key = os.getenv(self.API_KEY_NAME)
        if not api_key:
            self.logger.error(f"{self.API_KEY_NAME} not set")
            raise ValueError(f"{self.API_KEY_NAME} environment variable not set.")
        self.logger.info(f"{self.API_KEY_NAME} access successful")
        return api_key


config = Config()

[2025-06-11T11:45:15.294515] [INFO] [Config] Environment variables loaded


In [5]:
class Tools:
    """
    Utility class containing tools for financial data analysis.
    All methods are static as they don't require instance state.
    """

    @staticmethod
    def fetch_stock_data(ticker: str, period: str = "1y") -> dict:
        """
        Fetch stock data from Yahoo Finance.

        Args:
            ticker: Stock symbol (e.g., 'AAPL')
            period: Time period to fetch (default: '1y')

        Returns:
            Dictionary containing stock data with the following structure:
            {
                'ticker': str,
                'period': str,
                'data': {
                    'dates': List[str],
                    'open': List[float],
                    'high': List[float],
                    'low': List[float],
                    'close': List[float],
                    'volume': List[int]
                },
                'metadata': {
                    'current_price': float,
                    'price_change': float,
                    'volume': int,
                    'data_points': int
                }
            }

        Raises:
            ValueError: If ticker is invalid or data cannot be fetched
        """
        try:
            stock = yf.Ticker(ticker)
            data = stock.history(period=period)
            if data.empty:
                raise ValueError(f"No data found for ticker {ticker}")

            # Convert DataFrame to dictionary format
            result = {
                "ticker": ticker,
                "period": period,
                "data": {
                    "dates": data.index.strftime("%Y-%m-%d").tolist(),
                    "open": data["Open"].tolist(),
                    "high": data["High"].tolist(),
                    "low": data["Low"].tolist(),
                    "close": data["Close"].tolist(),
                    "volume": data["Volume"].tolist(),
                },
                "metadata": {
                    "current_price": float(data["Close"].iloc[-1]),
                    "price_change": float(
                        data["Close"].iloc[-1] - data["Close"].iloc[-2]
                    ),
                    "volume": int(data["Volume"].iloc[-1]),
                    "data_points": len(data),
                },
            }
            return result
        except Exception as e:
            raise ValueError(f"Error fetching data for {ticker}: {str(e)}")

    @staticmethod
    def calculate_technical_indicators(data: dict) -> dict:
        """
        Calculate basic technical indicators from stock data.

        Args:
            data: Dictionary containing stock data with OHLCV columns

        Returns:
            Dictionary containing calculated indicators
        """
        try:
            # Convert dictionary data back to DataFrame for calculations
            df = pd.DataFrame(
                {
                    "Close": data["data"]["close"],
                    "Open": data["data"]["open"],
                    "High": data["data"]["high"],
                    "Low": data["data"]["low"],
                    "Volume": data["data"]["volume"],
                },
                index=pd.to_datetime(data["data"]["dates"]),
            )

            # Calculate Simple Moving Averages
            df["SMA_20"] = df["Close"].rolling(window=20).mean()
            df["SMA_50"] = df["Close"].rolling(window=50).mean()

            # Calculate Relative Strength Index (RSI)
            delta = df["Close"].diff()
            gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
            loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
            rs = gain / loss
            df["RSI"] = 100 - (100 / (1 + rs))

            # Calculate MACD
            exp1 = df["Close"].ewm(span=12, adjust=False).mean()
            exp2 = df["Close"].ewm(span=26, adjust=False).mean()
            df["MACD"] = exp1 - exp2
            df["Signal_Line"] = df["MACD"].ewm(span=9, adjust=False).mean()

            # Get latest values
            latest = df.iloc[-1]

            return {
                "sma_20": float(latest["SMA_20"]),
                "sma_50": float(latest["SMA_50"]),
                "rsi": float(latest["RSI"]),
                "macd": float(latest["MACD"]),
                "signal_line": float(latest["Signal_Line"]),
                "current_price": float(latest["Close"]),
            }
        except Exception as e:
            raise ValueError(f"Error calculating technical indicators: {str(e)}")

    @staticmethod
    def generate_trading_signals(indicators: dict) -> str:
        """
        Generate trading signals based on technical indicators.

        Args:
            indicators: Dictionary containing technical indicators

        Returns:
            String containing trading signal and explanation
        """
        signals = []

        # RSI Analysis
        if indicators["rsi"] > 70:
            signals.append("RSI indicates overbought conditions")
        elif indicators["rsi"] < 30:
            signals.append("RSI indicates oversold conditions")

        # MACD Analysis
        if indicators["macd"] > indicators["signal_line"]:
            signals.append("MACD indicates bullish momentum")
        else:
            signals.append("MACD indicates bearish momentum")

        # Moving Average Analysis
        if indicators["sma_20"] > indicators["sma_50"]:
            signals.append("Short-term trend is bullish (SMA20 > SMA50)")
        else:
            signals.append("Short-term trend is bearish (SMA20 < SMA50)")

        # Combine signals into a readable format
        if signals:
            return "\n".join(signals)
        else:
            return "No clear trading signals detected"

In [6]:
# Create the Market Data Agent
market_data_agent = AssistantAgent(
    name="MarketDataAgent",
    description="Expert in fetching and processing financial market data",
    tools=[Tools.fetch_stock_data],  # Use the static method from Tools class
    model_client=OpenAIChatCompletionClient(
        model=config.MODEL_NAME, api_key=config.get_api_key()
    ),
    system_message="""You are a market data specialist. Your role is to:
    1. Fetch stock data using the fetch_stock_data tool
    2. Process and validate the data
    3. Provide clean, formatted data to other agents
    4. Handle any data fetching errors gracefully
    5. Ensure data completeness and quality
    
    When fetching data:
    - Always specify the ticker symbol
    - Use appropriate time periods (1d, 1wk, 1mo, 1y)
    - Verify the data is not empty
    - Check for missing values
    
    Always verify data quality before passing it on.""",
)

# Create the Technical Analysis Agent
technical_analysis_agent = AssistantAgent(
    name="TechnicalAnalysisAgent",
    description="Expert in technical analysis and market indicators",
    tools=[
        Tools.calculate_technical_indicators,
        Tools.generate_trading_signals,
    ],  # Add both technical analysis tools
    model_client=OpenAIChatCompletionClient(
        model=config.MODEL_NAME, api_key=config.get_api_key()
    ),
    system_message="""You are a technical analysis expert. Your role is to:
    1. Calculate technical indicators using calculate_technical_indicators
    2. Generate trading signals using generate_trading_signals
    3. Identify patterns and trends in the data
    4. Provide clear technical analysis insights
    5. Explain the significance of each indicator
    
    When analyzing:
    - Consider multiple timeframes
    - Look for confirmation across different indicators
    - Explain the reasoning behind signals
    - Highlight potential risks and limitations
    
    Always provide actionable insights with clear explanations.""",
)

# Create the User Proxy Agent
user_proxy = UserProxyAgent(
    name="UserProxy",
    description="An investment analysis assistant that coordinates between specialized agents",
)

[2025-06-11T11:45:15.314549] [INFO] [Config] Attempting to access GEMINI_API_KEY
[2025-06-11T11:45:15.314631] [INFO] [Config] GEMINI_API_KEY access successful
[2025-06-11T11:45:15.345608] [INFO] [Config] Attempting to access GEMINI_API_KEY
[2025-06-11T11:45:15.345638] [INFO] [Config] GEMINI_API_KEY access successful


In [7]:
async def analyze_stock(ticker: str) -> Dict:
    logger = Logger("StockAnalysis")
    logger.info(f"Starting analysis for ticker: {ticker}")

    try:
        if not ticker or not isinstance(ticker, str):
            raise ValueError("Invalid ticker symbol provided")

        # Create the group chat with the correct initialization
        groupchat = SelectorGroupChat(
            participants=[user_proxy, market_data_agent, technical_analysis_agent],
            model_client=OpenAIChatCompletionClient(
                model=config.MODEL_NAME, api_key=config.get_api_key()
            ),
            max_turns=10,
            selector_prompt="""You are in a role play game. The following roles are available:
{roles}.
Read the following conversation. Then select the next role from {participants} to play. Only return the role.

{history}

Read the above conversation. Then select the next role from {participants} to play. Only return the role.
""",
            allow_repeated_speaker=False,
            max_selector_attempts=3,
        )

        analysis_request = f"""Please analyze {ticker} stock comprehensively. Include:
        1. Current market data and price trends
        2. Technical indicators and their interpretation
        3. Trading signals and recommendations
        4. Risk assessment
        5. Summary of findings"""

        logger.info("Starting group chat analysis")
        chat_result = await groupchat.run(task=analysis_request)
        logger.info("Analysis completed successfully")

        return {
            "ticker": ticker,
            "analysis": chat_result,
            "timestamp": datetime.now().isoformat(),
            "status": "success",
            "error": None,
        }

    except Exception as e:
        error_msg = f"Analysis failed for {ticker}: {str(e)}"
        logger.error(error_msg)
        return {
            "ticker": ticker,
            "analysis": None,
            "timestamp": datetime.now().isoformat(),
            "status": "error",
            "error": error_msg,
        }

In [8]:
# Apply nest_asyncio to allow nested event loops in Jupyter
nest_asyncio.apply()


async def main():
    logger = Logger("Main")
    logger.info("Starting investment analysis chatbot")

    stocks_to_analyze = ["AAPL"]
    results = []

    try:
        for ticker in stocks_to_analyze:
            logger.info(f"Analyzing stock: {ticker}")
            result = await analyze_stock(ticker)

            if result["status"] == "success":
                logger.info(f"Successfully analyzed {ticker}")
                results.append(result)
            else:
                logger.error(f"Failed to analyze {ticker}: {result['error']}")

        print("\n=== Investment Analysis Results ===\n")

        for result in results:
            print(f"\nStock: {result['ticker']}")
            print(f"Timestamp: {result['timestamp']}")
            print("\nAnalysis:")
            print("-" * 50)
            print(result["analysis"])
            print("-" * 50)

    except Exception as e:
        logger.error(f"Error in main execution: {str(e)}")
        print(f"\nError: {str(e)}")

    finally:
        logger.info("Investment analysis chatbot completed")


# Run the example
if __name__ == "__main__":
    try:
        # Get the current event loop
        loop = asyncio.get_event_loop()
        # Run the main function
        loop.run_until_complete(main())
    except KeyboardInterrupt:
        print("\nAnalysis interrupted by user")
    except Exception as e:
        print(f"\nUnexpected error: {str(e)}")

[2025-06-11T11:45:15.446428] [INFO] [Main] Starting investment analysis chatbot
[2025-06-11T11:45:15.446526] [INFO] [Main] Analyzing stock: AAPL
[2025-06-11T11:45:15.446536] [INFO] [StockAnalysis] Starting analysis for ticker: AAPL
[2025-06-11T11:45:15.446543] [INFO] [Config] Attempting to access GEMINI_API_KEY
[2025-06-11T11:45:15.446552] [INFO] [Config] GEMINI_API_KEY access successful
[2025-06-11T11:45:15.455266] [INFO] [StockAnalysis] Starting group chat analysis
[2025-06-11T11:45:32.695233] [INFO] [StockAnalysis] Analysis completed successfully
[2025-06-11T11:45:32.695760] [INFO] [Main] Successfully analyzed AAPL

=== Investment Analysis Results ===


Stock: AAPL
Timestamp: 2025-06-11T11:45:32.695718

Analysis:
--------------------------------------------------
messages=[TextMessage(source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 6, 11, 4, 45, 15, 455421, tzinfo=datetime.timezone.utc), content='Please analyze AAPL stock comprehensively. Include:\n

Here’s a **human-readable summary** of what happened in this interaction step-by-step:

---

## 🧾 **User Request:**

At `2025-06-11 04:45:15 UTC`, the **user** asked:

> Please analyze AAPL stock comprehensively. Include:
>
> 1. Current market data and price trends
> 2. Technical indicators and their interpretation
> 3. Trading signals and recommendations
> 4. Risk assessment
> 5. Summary of findings

---

## 🤖 **Agents' Responses and Workflow:**

1. **MarketDataAgent** responded:

   > ❌ "I can only fetch stock data. I cannot analyze technical indicators, generate signals, or assess risk."

2. **TechnicalAnalysisAgent** responded:

   > ✅ "I can perform comprehensive analysis — but I need the stock data in OHLCV format (Open, High, Low, Close, Volume) first."

---

## 🤝 **Coordination Attempt:**

3. The system (via `UserProxy`) asked for input to proceed.

4. **MarketDataAgent** asked:

   > "Please provide the ticker and time period for the data."

5. The user (via system) provided:

   > `"AAPL, 1y"` — 1 year of Apple stock data.

---

## ⚙️ **Execution Attempt:**

6. **TechnicalAnalysisAgent** tried to run a function:

   > `calculate_technical_indicators(data={"AAPL": {"1y": {}}})`

7. ❌ **Error occurred**:

   > `"Error calculating technical indicators: 'data'"` — likely due to missing actual OHLCV data in the structure.

---

## 🔚 **Conclusion:**

- The process stopped because the **maximum number of allowed turns (10)** was reached.
- **No full analysis was completed**, because:

  - The technical analysis agent needed actual OHLCV data, but only got a placeholder or empty structure.
  - The MarketDataAgent never provided the full data in the required format.

---

## 📌 **What Went Wrong (Summary):**

| Step            | Issue                                                                        |
| --------------- | ---------------------------------------------------------------------------- |
| ✅ Request      | User gave a clear, structured stock analysis request                         |
| ⚠️ Agent roles  | One agent fetches data, the other analyzes — but they didn't coordinate well |
| ❌ Data missing | Data needed for technical analysis wasn’t passed correctly                   |
| ⏱ Timeout       | Conversation ended after 10 steps with no complete result                    |

---

## ✅ **How to Fix for Next Run:**

- Ensure MarketDataAgent returns actual OHLCV data (not an empty dict).
- Make sure the `data` argument passed to `calculate_technical_indicators()` is properly filled.
- Extend turn limit if needed for multi-agent workflows.

Let me know if you want a mock example of how the correct data structure should look.
